# SSH Key

Creamos SSH Key para poder usar GitHub sin necesidad de ingresar usuario y contraseña cada vez.

In [3]:
# Verificar GPU disponible
!nvidia-smi || echo "No se detectó GPU o nvidia-smi no está disponible."

Fri Jan  9 00:20:40 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
%%bash
set -e
GIT_EMAIL=${GIT_EMAIL:-"joaquin.arroyo100@gmail.com"}

# Genera llave efímera para esta sesión
if [ -f ~/.ssh/id_ed25519 ]; then
  echo "Ya existe ~/.ssh/id_ed25519; se reutiliza."
else
  ssh-keygen -t ed25519 -C "$GIT_EMAIL" -f ~/.ssh/id_ed25519 -N ""
fi

echo "Llave pública (pégala en GitHub → SSH Keys):"
cat ~/.ssh/id_ed25519.pub

Ya existe ~/.ssh/id_ed25519; se reutiliza.
Llave pública (pégala en GitHub → SSH Keys):
ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIPAX48wAhIkkK2t7RNt/+m4PTjrHg2LQH9aa9S7aZaKw 


In [9]:
# Clonar solo si falta, listar y actualizar (SSH)
import os

GIT_USER = "Joaquin Arroyo"
GIT_EMAIL = "joaquin.arroyo100@gmail.com"
GITHUB_USER = "joaquinarroyo"
REPO_NAME = "Tesina"
repo_path = REPO_NAME

os.environ["GIT_USER"] = GIT_USER
os.environ["GIT_EMAIL"] = GIT_EMAIL
os.environ["GITHUB_USER"] = GITHUB_USER
os.environ["REPO_NAME"] = REPO_NAME
os.environ["REPO_PATH"] = repo_path

cwd_name = os.path.basename(os.getcwd())

if cwd_name == REPO_NAME:
    print("Ya estás dentro del repo; se hace git pull.")
    !git remote set-url origin git@github.com:{GITHUB_USER}/{REPO_NAME}.git
    !git pull --rebase
else:
    if os.path.exists(repo_path):
        print(f"La carpeta {repo_path} ya existe; se hace git pull.")
        %cd {repo_path}
        !git remote set-url origin git@github.com:{GITHUB_USER}/{REPO_NAME}.git
        !git pull --rebase
    else:
        !git clone git@github.com:{GITHUB_USER}/{REPO_NAME}.git
        %cd {repo_path}

print(f"Contenido del directorio {repo_path}:")
%ls

Ya estás dentro del repo; se hace git pull.
Already up to date.
Contenido del directorio Tesina:
experiments/  LICENSE    requirements.txt  test.ipynb
helpers/      README.md  setup_mps.sh*


In [10]:
print("Instalando dependencias...")
!pip install -r requirements.txt

Instalando dependencias...
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121


In [12]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


### Ejecutar experimento secuencial
Ajusta los parámetros y lanza `experiments.sequential` desde la raíz del repo. Usa SSH y rutas locales (`runs/`).

In [13]:
# Configura y ejecuta experimento secuencial
# Ajusta parámetros según necesites
exp = "E3"
out_dir = "./runs"
repeat = 1
seed = 42
gpu_index = 0
no_plots = False
override_batch_size = None
override_epochs = None
override_lr = None
override_num_workers = None
override_amp = None  # usa 1 o 0 si quieres forzar

# Construye la línea de comando
cmd = [
    "python",
    "-m",
    "experiments.sequential",
    "--exp",
    exp,
    "--out",
    out_dir,
    "--repeat",
    str(repeat),
    "--seed",
    str(seed),
    "--gpu-index",
    str(gpu_index),
]

if no_plots:
    cmd.append("--no-plots")
if override_batch_size is not None:
    cmd.extend(["--override-batch-size", str(override_batch_size)])
if override_epochs is not None:
    cmd.extend(["--override-epochs", str(override_epochs)])
if override_lr is not None:
    cmd.extend(["--override-lr", str(override_lr)])
if override_num_workers is not None:
    cmd.extend(["--override-num-workers", str(override_num_workers)])
if override_amp is not None:
    cmd.extend(["--override-amp", str(int(bool(override_amp)))])

print("Ejecutando:", " ".join(cmd))
!{' '.join(cmd)}

Ejecutando: python -m experiments.sequential --exp E3 --out ./runs --repeat 1 --seed 42 --gpu-index 0
Ejecutando E3 | condition=sequential | seed=42 | pid=13386
Dataset: cifar10 | Modelo: resnet18 | bs=128 | epochs=5
Salida: runs/E3_cifar10_resnet18_sequential_20260109_002532_716485_pid13386
100% 170M/170M [00:03<00:00, 48.8MB/s] 
[cifar10 | resnet18] epoch 1/5 train_acc=0.439 val_acc=0.439 train_thr=1286.0 samp/s
[cifar10 | resnet18] epoch 2/5 train_acc=0.576 val_acc=0.603 train_thr=2828.7 samp/s
[cifar10 | resnet18] epoch 3/5 train_acc=0.638 val_acc=0.644 train_thr=2955.3 samp/s
[cifar10 | resnet18] epoch 4/5 train_acc=0.669 val_acc=0.671 train_thr=2797.7 samp/s
[cifar10 | resnet18] epoch 5/5 train_acc=0.701 val_acc=0.665 train_thr=2963.0 samp/s


In [15]:
!git add .
!git commit -m "Resultados del experimento {exp}" || echo "No hay cambios para commitear."
!git push origin main

[main a9e5ef1] Resultados del experimento E3
 20 files changed, 4070 insertions(+)
 create mode 100644 data/cifar-10-batches-py/batches.meta
 create mode 100644 data/cifar-10-batches-py/data_batch_1
 create mode 100644 data/cifar-10-batches-py/data_batch_2
 create mode 100644 data/cifar-10-batches-py/data_batch_3
 create mode 100644 data/cifar-10-batches-py/data_batch_4
 create mode 100644 data/cifar-10-batches-py/data_batch_5
 create mode 100644 data/cifar-10-batches-py/readme.html
 create mode 100644 data/cifar-10-batches-py/test_batch
 create mode 100644 data/cifar-10-python.tar.gz
 create mode 100644 runs/E3_cifar10_resnet18_sequential_20260109_002532_716485_pid13386/acc_vs_epoch.png
 create mode 100644 runs/E3_cifar10_resnet18_sequential_20260109_002532_716485_pid13386/epochs.csv
 create mode 100644 runs/E3_cifar10_resnet18_sequential_20260109_002532_716485_pid13386/gpu_power_over_time.png
 create mode 100644 runs/E3_cifar10_resnet18_sequential_20260109_002532_716485_pid13386/gpu_